In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import tensorflow as tf

# Important Note
*This is an artificial intelligence prediction application. It can make wrong guesses because of little data. If you believe you are at risk for a heart attack, consult your doctor instead of relying on this program.*

In [ ]:
CSV_COLUMN_NAMES = ["Age", "Sex", "RestingBloodPressure", "Cholesterol", "FastingBloodSugar", "MaxHeartRate", "Probability"]
PROBABILITY_HA = ["Low", "High"]

train = pd.read_csv("/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv")
train.drop(["cp", "restecg", "oldpeak", "slp", "caa", "thall", "exng"], axis = 1, inplace=True)
train.columns = CSV_COLUMN_NAMES
train_y = train.pop('Probability')

In [ ]:
# Input Func
def input_fn(features, labels, training, batch_size=256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

In [ ]:
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key, dtype=tf.uint32))

classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[300, 100],
    n_classes=3)

classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=10000)

In [ ]:
# Second Input Func
def input_fn2(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [ ]:
features = ["Age", "Sex", "RestingBloodPressure", "Cholesterol", "FastingBloodSugar", "MaxHeartRate"]
predict = {}

In [ ]:
# Get necessary data
print("Please type numeric values as prompted.")
for feature in features:
    valid = True
    while valid: 
        val = input(feature + ": ")
        if val.isdigit(): valid = False
    predict[feature] = [float(val)]

In [ ]:
# Write prediction
predictions = classifier.predict(input_fn=lambda: input_fn2(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    print('Prediction is "{}" ({:.1f}%)'.format(
        PROBABILITY_HA[class_id], 100 * probability))